In [4]:
import torch
from collections import Counter
import random
import re
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split



file_path = '/Users/jaafarbendriss/First-LLM/dataset/black_hole_wiki.txt'

# Open the file and read its content
with open(file_path, 'r') as file:
    file_content = file.read()

raw_text = file_content

def add_start_end_sentence(text):
    return "start-of-sentence " + text.replace("."," end-of-sentence start-of-sentence")


def remove_citations(text):
    pattern = r"\[\d+\]"
    cleaned_text = re.sub(pattern, "", text)    
    return cleaned_text

def remove_spaces(text):
    pattern = r"\s+"
    cleaned_text = re.sub(pattern, " ", text)    
    return cleaned_text

def preprocess(text: str):
    return remove_spaces(remove_citations(add_start_end_sentence(text).lower().replace(" s ","").replace("\\","").replace("'"," ").replace('"',' ').replace(","," ").replace("\ s"," ").replace("\n"," ")))

preprocessed_text = preprocess(raw_text)

Counter(preprocessed_text.split(" ")).most_common(20)

preprocessed_text_array = preprocessed_text.split(" ")


print(preprocessed_text_array[0:100])

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

tokens = tokenizer(preprocessed_text)

preprocessed_text = preprocessed_text.replace("end-of-sentence start-of-sentence", "end-of-sentence69696969start-of-sentence")
splited_data = preprocessed_text.split("69696969")

train_data =  [(i,sentence) for i, sentence in zip( range(0,len(splited_data)), splited_data  )]
train_iter = iter(train_data)


['start-of-sentence', 'object', 'that', 'has', 'a', 'no-return', 'boundary', 'for', 'other', 'uses', 'see', 'black', 'hole', '(disambiguation)', 'end-of-sentence', 'start-of-sentence', 'direct', 'radio', 'image', 'of', 'a', 'supermassive', 'black', 'hole', 'at', 'the', 'core', 'of', 'messier', '87', 'animated', 'simulation', 'of', 'a', 'schwarzschild', 'black', 'hole', 'with', 'a', 'galaxy', 'passing', 'behind', 'end-of-sentence', 'start-of-sentence', 'around', 'the', 'time', 'of', 'alignment', 'extreme', 'gravitational', 'lensing', 'of', 'the', 'galaxy', 'is', 'observed', 'end-of-sentence', 'start-of-sentence', 'a', 'black', 'hole', 'is', 'a', 'region', 'of', 'spacetime', 'where', 'gravity', 'is', 'so', 'strong', 'that', 'nothing', 'including', 'light', 'and', 'other', 'electromagnetic', 'waves', 'has', 'enough', 'energy', 'to', 'escape', 'it', 'end-of-sentence', 'start-of-sentence', 'einstein', 's', 'theory', 'of', 'general', 'relativity', 'predicts', 'that', 'a', 'sufficiently', 'co

In [5]:
next(train_iter)

(0,
 'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence')

## Preparing training dataset using sliding window

In [6]:
sentence =  'start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence'

In [7]:
data = sentence.split()

In [8]:
splited_data[0:2]

['start-of-sentence object that has a no-return boundary for other uses see black hole (disambiguation) end-of-sentence',
 'start-of-sentence direct radio image of a supermassive black hole at the core of messier 87 animated simulation of a schwarzschild black hole with a galaxy passing behind end-of-sentence']

In [9]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
vocab(tokens[0:10])

[4, 37, 10, 66, 6, 0, 186, 16, 68, 0]

In [10]:
token_map = vocab.get_stoi()

In [11]:
splited_data[20]

'start-of-sentence the presence of a black hole can be inferred through its interaction with other matter and with electromagnetic radiation such as visible light end-of-sentence'

In [12]:
training_data = []

for sentence in splited_data:
    data=sentence.split()
    for i in range(1,len(data)):
        X=data[0:i]
        y = data[i]
        training_data.append((X,y))

In [13]:
len(training_data)

11538

In [14]:
training_data[20:21]

[(['start-of-sentence', 'direct', 'radio', 'image', 'of', 'a', 'supermassive'],
  'black')]

In [15]:
# second method

training_data_cut = []

for sentence in splited_data:
    data=sentence.split()
    for i in range(1,len(data)):
        X=data[max([0,i-5]):i]
        y = data[i]
        training_data_cut.append((X,y))

In [16]:
training_data_cut[20:21]

[(['radio', 'image', 'of', 'a', 'supermassive'], 'black')]

In [17]:
word_to_token = token_map

def word_to_token_wrapper(word: str) -> int:
    if word not in word_to_token:
        return word_to_token["<unk>"]
    return word_to_token[word]

def list_sentence_to_token(sentence_list)->int:
    return [word_to_token_wrapper(word) for word in sentence_list]

def sentence_to_token(sentence):
    return [word_to_token_wrapper(word) for word in sentence.split()]


In [18]:
sentence_to_token("black")

[5]

In [19]:
word_to_token["<unk>"]

0

In [20]:
training_data_cut[4]

(['start-of-sentence', 'object', 'that', 'has', 'a'], 'no-return')

In [21]:
training_data_tokenized = [ (list_sentence_to_token(a),word_to_token_wrapper(b) ) for a,b in training_data_cut]

In [22]:
training_data_tokenized[110:115]

[([186, 2, 92, 138, 12], 115),
 ([2, 92, 138, 12, 115], 1),
 ([92, 138, 12, 115, 1], 26),
 ([138, 12, 115, 1, 26], 29),
 ([12, 115, 1, 26, 29], 3)]

## training 

In [23]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

class SentenceCompletionDataset(Dataset):
    def __init__(self, data, word_to_index):
        self.data = data
        self.word_to_index = word_to_index
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_sequence, target_word = self.data[idx]
        input_indices = [self.word_to_index[word] for word in input_sequence]
        target_index = self.word_to_index[target_word]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_index, dtype=torch.long)



In [24]:

# Assuming train_data is a list of sentences
train_sentences, val_sentences = train_test_split(training_data_tokenized, test_size=0.3, random_state=42)

# Now, apply the masking and preparation logic to both train_sentences and val_sentences

In [25]:

class TokenDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        tokens, target = self.data[idx]
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(target, dtype=torch.long)

# Assuming `data` is your list of mappings
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

In [26]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    tokens, targets = zip(*batch)
    # Pad the sequences to the maximum length in the batch
    tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)
    targets = torch.tensor(targets, dtype=torch.long)
    return tokens_padded, targets


In [27]:
len(train_sentences)

8076

In [28]:
vocab_size = len(word_to_token)
vocab_size

2147

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,dropout_rate=0.5):
        super(SimpleLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)  # Add dropout layer

        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, tokens):
        embedded = self.embedding(tokens)  # [batch_size, seq_len, embedding_dim]
        # Aggregate embeddings, e.g., by averaging
        aggregated = embedded.mean(dim=1)  # [batch_size, embedding_dim]
        hidden = self.relu(self.linear1(aggregated))  # [batch_size, hidden_dim]
        output = self.linear2(hidden)  # [batch_size, output_dim]
        return output

# Hyperparameters
#vocab_size = 2145  # Adjust based on your dataset
embedding_dim = 50
hidden_dim = 20
output_dim = vocab_size  # Same as vocab size for prediction

model = SimpleLM(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
len(train_sentences)

8076

In [31]:
data = train_sentences

In [32]:
dataset = TokenDataset(data)
dataloader = DataLoader(dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)


In [33]:
num_epochs = 150  # Or whatever suits your dataset

for epoch in range(num_epochs):
    total_loss = 0
    for tokens, target in dataloader:
        optimizer.zero_grad()
        output = model(tokens)
        loss = criterion(output, target.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    if epoch %10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')

# batch 5 => Epoch 141, Loss: 0.3472299749050117
# batch 7 => Epoch 141, Loss: 0.3254897424296818

# batch 5 and window 5 => Epoch 141, Loss: 0.2996615952255913
"""
embedding_dim = 80
hidden_dim = 50
0.26723970297246136
"""

"""
embedding_dim = 40
hidden_dim = 20
0.37902324068592874
"""


"""
embedding_dim = 50
hidden_dim = 20
dropout = 0.5
0.3662537602579914
"""

/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_74746/763346120.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens_padded = pad_sequence([torch.tensor(seq) for seq in tokens], batch_first=True, padding_value=0)


Epoch 1, Loss: 6.4738432998114295
Epoch 11, Loss: 3.7057437417235706
Epoch 21, Loss: 2.3452130593728313
Epoch 31, Loss: 1.7310304481185752
Epoch 41, Loss: 1.3395805589930792
Epoch 51, Loss: 1.0698184460098394
Epoch 61, Loss: 0.8778534088267064
Epoch 71, Loss: 0.7375941460294156
Epoch 81, Loss: 0.6320147079432621
Epoch 91, Loss: 0.5492831248257513
Epoch 101, Loss: 0.48809121044883114
Epoch 111, Loss: 0.44342492915242393
Epoch 121, Loss: 0.4098725147817182
Epoch 131, Loss: 0.38513790079194016
Epoch 141, Loss: 0.36309760622634996


'\nembedding_dim = 50\nhidden_dim = 20\ndropout = 0.5\n0.3662537602579914\n'

In [34]:
[([4, 1, 186, 2, 92, 138, 12], 115),
 ([4, 1, 186, 2, 92, 138, 12, 115], 1),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1], 26),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26], 29),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29], 3)]

[([4, 1, 186, 2, 92, 138, 12], 115),
 ([4, 1, 186, 2, 92, 138, 12, 115], 1),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1], 26),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26], 29),
 ([4, 1, 186, 2, 92, 138, 12, 115, 1, 26, 29], 3)]

In [35]:
train_sentences[10:15]

[([2, 1, 110, 160, 36], 34),
 ([147, 6, 2052, 20, 766], 71),
 ([50, 1069, 190, 7, 1], 410),
 ([42, 70, 115, 794, 42], 98),
 ([1370, 300, 64, 57, 53], 3)]

In [36]:
seq = sentence_to_token("the black")
seq

[1, 5]

In [37]:
token_to_word = {token: word for word, token in word_to_token.items()}

def token_to_sentence(tokens):
    return " ".join([token_to_word[t] for t in tokens])

In [38]:
token_to_sentence([2, 1, 110, 160, 36])

'of the photon orbit which'

In [39]:
sequence = [2, 1, 110, 160, 36]
sequence_tensor = torch.tensor(sequence).unsqueeze(0) 
sequence_tensor

tensor([[  2,   1, 110, 160,  36]])

In [40]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    output = model(sequence_tensor)
    predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
predicted_token_id

34

In [41]:
token_to_word[predicted_token_id]

'can'

In [42]:
TOPK = 1

def predict_missing_word(sentence="black hole is the biggest object in the"):
    last_K = " ".join(sentence.split()[-5:])
    #print(last_K)                  
    sequence = sentence_to_token(last_K)
    sequence_tensor = torch.tensor(sequence).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        output = model(sequence_tensor)
        predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
        #_, top5_indices = torch.topk(output[0], TOPK)
        #predicted_token_id = random.choice(top5_indices.tolist())

    # Assuming you have a token_to_word dictionary
    token_to_word = {token: word for word, token in word_to_token.items()}
    predicted_word = token_to_word[predicted_token_id]
    return predicted_word

In [43]:
def predict_missing_token(tokens):
               
    sequence = tokens[-5:]
    sequence_tensor = torch.tensor(sequence).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        output = model(sequence_tensor)
        predicted_token_id = output.argmax(dim=1).item()  # Get the index of the max log-probability
        #_, top5_indices = torch.topk(output[0], TOPK)
        #predicted_token_id = random.choice(top5_indices.tolist())

    return predicted_token_id

In [44]:
predict_missing_word()

'disk'

In [45]:
next_word = "start-of-sentence"
#next_word = "einstein"
words = []
sentence = next_word
words.append(next_word)
limit = 0
while next_word != "end-of-sentence" and limit<20:
    #print("input ",sentence)
    next_word = predict_missing_word(sentence)
    #print("output ",next_word)
    words.append(next_word)
    sentence = " ".join(words)
    limit += 1
words.append(next_word)

" ".join(words)

'start-of-sentence [ edit ] main <unk> hawking radiation smaller of in and in sagittarius cannot resolve slowed better large density rotating rotating'

In [46]:
correct = 0

for s in val_sentences:
    #print(s)
    predicted_token = predict_missing_token(s[0]) 
    #print("predicted token = ", predicted_token)
    answer = predicted_token == s[1]

    if answer:
        correct += answer

print()
print(correct,len(val_sentences))
"""
embedding_dim = 80
hidden_dim = 50
259 3462
"""
# 



203 3462


'\nembedding_dim = 80\nhidden_dim = 50\n259 3462\n'

In [47]:
correct = 0

for s in train_sentences:
    #print(s)
    predicted_token = predict_missing_token(s[0]) 
    #print("predicted token = ", predicted_token)
    answer = predicted_token == s[1]

    if answer:
        correct += answer

print()
print(correct,len(train_sentences))
"""
embedding_dim = 80
hidden_dim = 50
7261 8076
"""



7114 8076


'\nembedding_dim = 80\nhidden_dim = 50\n7261 8076\n'